# Chapter 19: Project Configuration with pyproject.toml

This notebook covers the modern standard for configuring Python projects using
`pyproject.toml` (PEP 621). We explore project metadata, build systems, entry points,
optional dependencies, project layouts, and runtime metadata inspection.

## Key Concepts
- **pyproject.toml**: The single configuration file for modern Python projects
- **PEP 621**: Standardized project metadata in `[project]` table
- **Build backends**: Pluggable systems that turn source into installable packages
- **Entry points**: Mechanism for registering CLI tools and plugins

## Section 1: pyproject.toml -- The Modern Standard

Before `pyproject.toml`, Python projects used a patchwork of configuration files:
`setup.py`, `setup.cfg`, `MANIFEST.in`, `requirements.txt`, and tool-specific files
like `mypy.ini`, `pytest.ini`, `.flake8`, etc.

**pyproject.toml** (introduced in PEP 518, refined by PEP 621) consolidates project
metadata and tool configuration into a single, declarative TOML file.

### Minimal Example

```toml
[project]
name = "my-package"
version = "1.0.0"
requires-python = ">=3.10"
dependencies = [
    "requests>=2.28",
    "click>=8.0",
]

[build-system]
requires = ["setuptools>=68.0"]
build-backend = "setuptools.build_meta"
```

In [ ]:
# Python 3.11+ includes tomllib for reading TOML files
# For older versions, use the third-party 'tomli' package
try:
    import tomllib  # Python 3.11+
except ModuleNotFoundError:
    import tomli as tomllib  # type: ignore[no-redef]

# Parse a pyproject.toml string to understand its structure
sample_toml: str = """
[project]
name = "example-cli"
version = "2.1.0"
description = "A demonstration CLI tool"
requires-python = ">=3.10"
license = "MIT"
authors = [
    {name = "Jane Developer", email = "jane@example.com"},
]
dependencies = [
    "click>=8.0",
    "rich>=13.0",
]

[project.optional-dependencies]
dev = ["pytest>=7.0", "mypy>=1.0"]
docs = ["sphinx>=7.0", "furo"]

[project.scripts]
example-cli = "example_cli.main:app"

[build-system]
requires = ["setuptools>=68.0"]
build-backend = "setuptools.build_meta"
"""

config: dict = tomllib.loads(sample_toml)

print("Parsed [project] table:")
project = config["project"]
for key, value in project.items():
    print(f"  {key}: {value}")

## Section 2: The `[project]` Table (PEP 621)

The `[project]` table contains all the metadata about your package:

| Field | Required? | Description |
|---|---|---|
| `name` | Yes | Package name (used on PyPI) |
| `version` | Yes* | Semantic version string |
| `description` | No | One-line summary |
| `requires-python` | Recommended | Minimum Python version |
| `dependencies` | Recommended | Runtime dependencies |
| `license` | Recommended | SPDX license identifier |
| `authors` | No | List of `{name, email}` tables |
| `readme` | No | Path to README file |
| `classifiers` | No | PyPI trove classifiers |

\* Can be dynamic if set via `[project.dynamic]`

In [ ]:
from dataclasses import dataclass, field


@dataclass
class ProjectMetadata:
    """Represents the [project] table of pyproject.toml."""
    name: str
    version: str
    description: str = ""
    requires_python: str = ">=3.10"
    dependencies: list[str] = field(default_factory=list)
    optional_dependencies: dict[str, list[str]] = field(default_factory=dict)
    scripts: dict[str, str] = field(default_factory=dict)

    @classmethod
    def from_toml(cls, config: dict) -> "ProjectMetadata":
        """Parse a [project] table dict into a ProjectMetadata instance."""
        project: dict = config["project"]
        return cls(
            name=project["name"],
            version=project["version"],
            description=project.get("description", ""),
            requires_python=project.get("requires-python", ">=3.10"),
            dependencies=project.get("dependencies", []),
            optional_dependencies=project.get("optional-dependencies", {}),
            scripts=project.get("scripts", {}),
        )


metadata = ProjectMetadata.from_toml(config)
print(f"Package:      {metadata.name}")
print(f"Version:      {metadata.version}")
print(f"Description:  {metadata.description}")
print(f"Requires:     Python {metadata.requires_python}")
print(f"Dependencies: {metadata.dependencies}")
print(f"Scripts:      {metadata.scripts}")
print(f"Extras:       {list(metadata.optional_dependencies.keys())}")

## Section 3: The `[build-system]` Table

The `[build-system]` table tells tools like `pip` and `build` how to build your package.

| Backend | Package | Strengths |
|---|---|---|
| **setuptools** | `setuptools>=68.0` | Mature, widely used, highly configurable |
| **hatchling** | `hatchling` | Modern, fast, good defaults |
| **flit-core** | `flit-core>=3.4` | Minimal, pure-Python only |
| **poetry-core** | `poetry-core>=1.0` | Used with Poetry workflow |
| **maturin** | `maturin>=1.0` | For Rust+Python (PyO3) extensions |

### Example configurations for each backend

```toml
# setuptools
[build-system]
requires = ["setuptools>=68.0", "wheel"]
build-backend = "setuptools.build_meta"

# hatchling
[build-system]
requires = ["hatchling"]
build-backend = "hatchling.build"

# flit
[build-system]
requires = ["flit_core>=3.4"]
build-backend = "flit_core.api"

# poetry
[build-system]
requires = ["poetry-core>=1.0.0"]
build-backend = "poetry.core.masonry.api"
```

In [ ]:
# Comparing build system configurations
from dataclasses import dataclass


@dataclass
class BuildSystem:
    """Represents the [build-system] table."""
    requires: list[str]
    build_backend: str

    @property
    def backend_name(self) -> str:
        """Extract the human-readable backend name."""
        mapping: dict[str, str] = {
            "setuptools.build_meta": "setuptools",
            "hatchling.build": "hatchling",
            "flit_core.api": "flit",
            "poetry.core.masonry.api": "poetry",
        }
        return mapping.get(self.build_backend, self.build_backend)


build_configs: list[BuildSystem] = [
    BuildSystem(["setuptools>=68.0", "wheel"], "setuptools.build_meta"),
    BuildSystem(["hatchling"], "hatchling.build"),
    BuildSystem(["flit_core>=3.4"], "flit_core.api"),
    BuildSystem(["poetry-core>=1.0.0"], "poetry.core.masonry.api"),
]

print(f"{'Backend':<15} {'Build Requires':<35} {'build-backend'}")
print("-" * 75)
for bs in build_configs:
    requires_str = ", ".join(bs.requires)
    print(f"{bs.backend_name:<15} {requires_str:<35} {bs.build_backend}")

## Section 4: Entry Points -- Console Scripts

Entry points let you register executable commands that get installed into the
environment's `bin/` (or `Scripts/`) directory.

```toml
[project.scripts]
my-tool = "my_package.cli:main"
```

This means: when the user types `my-tool` in their terminal, Python calls
`my_package.cli.main()`. The format is `module_path:callable`.

### GUI Scripts
```toml
[project.gui-scripts]
my-gui = "my_package.gui:launch"
```
On Windows, `gui-scripts` suppresses the console window.

### Plugin Entry Points
```toml
[project.entry-points."myapp.plugins"]
csv-handler = "myapp.plugins.csv:CsvPlugin"
json-handler = "myapp.plugins.json:JsonPlugin"
```
This is the standard mechanism for **plugin discovery** in Python.

In [ ]:
from dataclasses import dataclass


@dataclass
class EntryPoint:
    """Represents a console_scripts entry point."""
    command: str
    module: str
    callable: str

    @classmethod
    def parse(cls, command: str, reference: str) -> "EntryPoint":
        """Parse an entry point from 'module.path:callable' format."""
        module, func = reference.split(":")
        return cls(command=command, module=module, callable=func)

    def to_import_statement(self) -> str:
        return f"from {self.module} import {self.callable}"

    def __repr__(self) -> str:
        return f"{self.command} -> {self.module}:{self.callable}"


# Parse the entry point from our sample config
scripts: dict[str, str] = config["project"]["scripts"]

for cmd, ref in scripts.items():
    ep = EntryPoint.parse(cmd, ref)
    print(f"Command:    {ep.command}")
    print(f"Module:     {ep.module}")
    print(f"Callable:   {ep.callable}")
    print(f"Import:     {ep.to_import_statement()}")
    print()
    print(f"When a user runs `{ep.command}`, Python executes:")
    print(f"  {ep.to_import_statement()}")
    print(f"  {ep.callable}()")

## Section 5: Optional Dependencies and Extras

Optional dependencies let users install additional features only when needed:

```toml
[project.optional-dependencies]
dev = ["pytest>=7.0", "mypy>=1.0", "ruff>=0.1"]
docs = ["sphinx>=7.0", "furo"]
postgres = ["psycopg2>=2.9"]
```

Users install extras with square bracket syntax:
```bash
pip install my-package[dev]          # Install dev extras
pip install my-package[dev,docs]     # Install multiple extras
pip install -e ".[dev]"              # Editable install with extras
```

In [ ]:
# Inspecting optional dependencies from our parsed config
optional_deps: dict[str, list[str]] = config["project"].get(
    "optional-dependencies", {}
)

print("Available extras:")
for extra_name, deps in optional_deps.items():
    print(f"\n  [{extra_name}]")
    for dep in deps:
        print(f"    - {dep}")

# Show the install commands
package_name: str = config["project"]["name"]
print(f"\nInstall commands:")
for extra_name in optional_deps:
    print(f"  pip install {package_name}[{extra_name}]")

all_extras: str = ",".join(optional_deps.keys())
print(f"  pip install {package_name}[{all_extras}]  # all extras")

## Section 6: src-Layout vs Flat Layout

Two common ways to organize your project source code:

### Flat Layout
```
my-project/
  my_package/
    __init__.py
    core.py
  tests/
    test_core.py
  pyproject.toml
```

### src-Layout (Recommended)
```
my-project/
  src/
    my_package/
      __init__.py
      core.py
  tests/
    test_core.py
  pyproject.toml
```

| Aspect | Flat Layout | src-Layout |
|---|---|---|
| **Import safety** | Can accidentally import uninstalled code | Forces you to install first |
| **Test isolation** | Tests may use local code, not installed | Tests always use installed package |
| **Simplicity** | Fewer directories | One extra `src/` level |
| **setuptools config** | Automatic discovery | Needs `[tool.setuptools.packages.find]` |

In [ ]:
from pathlib import Path


def display_layout(name: str, structure: dict) -> None:
    """Display a project directory structure."""
    print(f"\n{name}:")
    _print_tree(structure, prefix="  ")


def _print_tree(tree: dict, prefix: str = "") -> None:
    """Recursively print a directory tree."""
    items = list(tree.items())
    for i, (name, children) in enumerate(items):
        is_last: bool = i == len(items) - 1
        connector: str = "└── " if is_last else "├── "
        print(f"{prefix}{connector}{name}")
        if isinstance(children, dict):
            extension = "    " if is_last else "│   "
            _print_tree(children, prefix + extension)


# Flat layout
flat: dict = {
    "my_package/": {
        "__init__.py": None,
        "core.py": None,
        "cli.py": None,
    },
    "tests/": {
        "test_core.py": None,
        "test_cli.py": None,
    },
    "pyproject.toml": None,
    "README.md": None,
}

# src-layout
src_layout: dict = {
    "src/": {
        "my_package/": {
            "__init__.py": None,
            "core.py": None,
            "cli.py": None,
        },
    },
    "tests/": {
        "test_core.py": None,
        "test_cli.py": None,
    },
    "pyproject.toml": None,
    "README.md": None,
}

display_layout("Flat Layout", flat)
display_layout("src-Layout", src_layout)

In [ ]:
# setuptools configuration for each layout

flat_config: str = """\
# Flat layout -- setuptools auto-discovers packages
[tool.setuptools.packages.find]
exclude = ["tests*"]
"""

src_config: str = """\
# src-layout -- tell setuptools where to find packages
[tool.setuptools.packages.find]
where = ["src"]
"""

print("setuptools configuration:")
print(flat_config)
print(src_config)

## Section 7: `importlib.metadata` -- Reading Package Metadata at Runtime

The `importlib.metadata` module (Python 3.8+) lets you inspect installed package
metadata at runtime -- version numbers, entry points, dependencies, and more.
This is the standard way to implement `--version` flags in CLI tools.

In [ ]:
from importlib.metadata import (
    metadata,
    version,
    requires,
    packages_distributions,
)

# Get the version of an installed package
pip_version: str = version("pip")
print(f"pip version: {pip_version}")

# Get full metadata for a package
pip_meta = metadata("pip")
print(f"\nPackage metadata for 'pip':")
print(f"  Name:       {pip_meta['Name']}")
print(f"  Version:    {pip_meta['Version']}")
print(f"  Summary:    {pip_meta['Summary']}")
print(f"  Home-page:  {pip_meta.get('Home-page', 'N/A')}")
print(f"  License:    {pip_meta.get('License', 'N/A')}")

# Get the dependencies of a package
pip_deps: list[str] | None = requires("pip")
if pip_deps:
    print(f"\nDependencies ({len(pip_deps)} total):")
    for dep in pip_deps[:5]:
        print(f"  - {dep}")
    if len(pip_deps) > 5:
        print(f"  ... and {len(pip_deps) - 5} more")
else:
    print("\nNo dependencies listed.")

In [ ]:
from importlib.metadata import entry_points, version, PackageNotFoundError

# Discover console_scripts entry points installed in this environment
# entry_points() returns a dict-like object keyed by group name
console_eps = entry_points(group="console_scripts")

print(f"Console scripts in this environment: {len(list(console_eps))}")
print()

for ep in list(console_eps)[:8]:
    print(f"  {ep.name:25s} -> {ep.value}")

if len(list(console_eps)) > 8:
    print(f"  ... and {len(list(console_eps)) - 8} more")

# Practical example: getting your own package version
def get_version(package_name: str) -> str:
    """Get the installed version of a package, or 'unknown'."""
    try:
        return version(package_name)
    except PackageNotFoundError:
        return "unknown (not installed)"


print(f"\npip version:        {get_version('pip')}")
print(f"setuptools version: {get_version('setuptools')}")
print(f"nonexistent:        {get_version('nonexistent-package')}")

## Section 8: `sysconfig` -- Understanding Installation Paths

The `sysconfig` module reveals where Python installs files. This is essential for
understanding how packages are laid out on disk and for debugging import issues.

In [ ]:
import sysconfig

# Get the installation scheme name
scheme: str = sysconfig.get_default_scheme()
print(f"Default installation scheme: {scheme}")
print()

# All known paths in the current scheme
paths: dict[str, str] = sysconfig.get_paths()
print("Installation paths:")
for name, path in sorted(paths.items()):
    print(f"  {name:15s} -> {path}")

print(f"\nPlatform:     {sysconfig.get_platform()}")
print(f"Python path:  {sysconfig.get_python_lib()}")

In [ ]:
import sysconfig

# sysconfig also exposes build-time configuration variables
# These are the values from when the Python interpreter was compiled

interesting_vars: list[str] = [
    "prefix",
    "exec_prefix",
    "py_version",
    "py_version_short",
    "SOABI",          # Shared object ABI tag
    "EXT_SUFFIX",     # Extension module suffix (e.g. .cpython-312-darwin.so)
]

print("Selected configuration variables:")
for var in interesting_vars:
    value = sysconfig.get_config_var(var)
    print(f"  {var:20s} = {value}")

# Available installation schemes
print(f"\nAvailable schemes: {sysconfig.get_scheme_names()}")

## Summary

### pyproject.toml
- **`[project]`**: Standardized metadata (PEP 621) -- name, version, dependencies
- **`[build-system]`**: Declares the build backend (setuptools, hatchling, flit, poetry)
- **`[project.scripts]`**: Register CLI commands via `module:callable` references
- **`[project.optional-dependencies]`**: Define extras installable with `pip install pkg[extra]`

### Project Layout
- **src-layout** is recommended: forces installation before import, better test isolation
- **Flat layout** is simpler but can mask import errors during development

### Runtime Introspection
- **`importlib.metadata`**: Read installed package versions, metadata, and entry points
- **`sysconfig`**: Inspect installation paths, platform info, and build configuration
- Use `importlib.metadata.version()` for `--version` flags in CLI tools